In [89]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [474]:
import xgboost as xgb
import pandas as pd
import numpy as np
pd.options.display.max_columns = 999
pd.options.display.max_rows = 50

In [191]:
import ReadFromDB as RDB

In [497]:
def station_function(i):
    """
    Creates a sparse matrix from station information on database
    """
    
    station = RDB.add_times(RDB.create_station_dictionary(i)[i])
    weather = RDB.weather_times(RDB.read_weather(i))
    df = pd.merge(station, weather, left_on=['date', 'hour', 'first_half_hour', 'second_half_hour'], right_on=['date', 'hour', 'first_half_hour', 'second_half_hour'], how='inner')
    df.drop_duplicates(subset='last_updated', inplace=True)
    df.drop_duplicates(subset='time', inplace=True)

    df = df.reset_index()
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    weather_icons = ['partly-cloudy-day',
                     'partly-cloudy-night',
                     'clear-night',
                     'clear-day',
                     'fog',
                     'wind',
                     'cloudy',
                     'rain'
                    ]
    weather_cols = ['apparentTemperature',
                    'cloudCover',
                    'dewPoint',
                    'humidity',
                    'precipIntensity',
                    'precipProbability',
                    'temperature',
                    'windGust',
                    'windSpeed'
                    ]
    
    for day in days:
        df[day] = 0
    
    for icon in weather_icons:
        df[icon] = 0
    
    for j in df.index:
        val = df.loc[j, 'day_x']
        for day in days:
            if val == day:
                df.loc[j, day] = 1
        
        icon_val = df.loc[j, 'icon']
        for icon in weather_icons:
            if icon_val == icon:
                df.loc[j, icon] = 1
     
    data = pd.DataFrame((df.Mon, df.Tue, df.Wed, df.Thu, df.Fri, df.Sat, df.Sun, df.hour, df.first_half_hour, df.second_half_hour)).T
    
    for icon in weather_icons:
        data[icon] = df[icon]
    for col in weather_cols:
        data[col] = df[col]
   
    label = pd.DataFrame((df.available_bikes))
    
    return [df, data, label]

In [501]:
%%time
[station, data, label] = station_function(2)

Wall time: 10.6 s


In [503]:
%%time 
label

Wall time: 0 ns


,available_bikes
0,3
1,4
2,3
3,3
4,1
5,3
6,2
7,5
8,3
9,7


In [437]:
station_2 = RDB.add_times(RDB.create_station_dictionary(2)[2])

In [384]:
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for day in days:
    station_2[day] = 0

In [385]:
for i in station_2.index:
    val = station_2.loc[i, 'day']
    for day in days:
        if val == day:
            station_2.loc[i, day] = 1

In [460]:
station_2

,status,bike_stands,available_bike_stands,available_bikes,last_updated,date,day,hour,minute,second,first_half_hour,second_half_hour
0,OPEN,20,19,1,1550592601,02/19/19,Tue,16,10,1,True,False
1,OPEN,20,20,0,1550593147,02/19/19,Tue,16,19,7,True,False
2,OPEN,20,19,1,1550593810,02/19/19,Tue,16,30,10,False,True
3,OPEN,20,19,1,1550594419,02/19/19,Tue,16,40,19,False,True
4,OPEN,20,20,0,1550595287,02/19/19,Tue,16,54,47,False,True
5,OPEN,20,20,0,1550595896,02/19/19,Tue,17,4,56,True,False
6,OPEN,20,19,1,1550596280,02/19/19,Tue,17,11,20,True,False
7,OPEN,20,19,1,1550596889,02/19/19,Tue,17,21,29,True,False
8,OPEN,20,19,1,1550597758,02/19/19,Tue,17,35,58,False,True
9,OPEN,20,20,0,1550597820,02/19/19,Tue,17,37,0,False,True


In [299]:
# %%time
# for i in range(24):
#     station_2['hour_{}'.format(i)] = 0

In [300]:
# %%time
# for i in station_2.index:
#     hour_val = station_2.loc[i, 'hour']
#     for k in range(24):
#         if hour_val == k:
#             station_2.loc[i, 'hour_{}'.format(k)] = 1

In [388]:
new_df = pd.DataFrame((station_2.available_bikes,
                     station_2.date,
                     station_2.Mon,
                     station_2.Tue,
                     station_2.Wed,
                     station_2.Thu,
                     station_2.Fri,
                     station_2.Sat,
                     station_2.Sun,
                     station_2.hour,
                     station_2.first_half_hour,
                     station_2.second_half_hour)).T

new_df.drop_duplicates(inplace=True)

data = pd.DataFrame((new_df.Mon,
                     new_df.Tue,
                     new_df.Wed,
                     new_df.Thu,
                     new_df.Fri,
                     new_df.Sat,
                     new_df.Sun,
                     new_df.hour,
                     new_df.first_half_hour,
                     new_df.second_half_hour)).T

for col in data.columns:
    data[col] = data[col].apply(int)

    
label = pd.DataFrame((new_df.available_bikes))

for col in label.columns:
    label[col] = label[col].apply(int)


In [504]:
data.dtypes

Mon                      int64
Tue                      int64
Wed                      int64
Thu                      int64
Fri                      int64
Sat                      int64
Sun                      int64
hour                     int64
first_half_hour          int64
second_half_hour         int64
partly-cloudy-day        int64
partly-cloudy-night      int64
clear-night              int64
clear-day                int64
fog                      int64
wind                     int64
cloudy                   int64
rain                     int64
apparentTemperature    float64
cloudCover             float64
dewPoint               float64
humidity               float64
precipIntensity        float64
precipProbability      float64
temperature            float64
windGust               float64
windSpeed              float64
dtype: object

In [512]:
dtrain = xgb.DMatrix(data[:1000], label=label[:1000])
dtest = xgb.DMatrix(data[1000:], label=label[1000:])

In [518]:
param = {}
watchlist = [(dtest, 'eval'), (dtrain, 'train')]
num_round = 10
bst = xgb.train(param, dtrain, num_round, watchlist)
preds = bst.predict(dtest)
labels = dtest.get_label()

[0]	eval-rmse:6.25368	train-rmse:5.8369
[1]	eval-rmse:5.13479	train-rmse:4.48735
[2]	eval-rmse:4.59608	train-rmse:3.53048
[3]	eval-rmse:4.29516	train-rmse:2.88324
[4]	eval-rmse:4.15534	train-rmse:2.44143
[5]	eval-rmse:4.07809	train-rmse:2.15021
[6]	eval-rmse:4.01963	train-rmse:1.95277
[7]	eval-rmse:3.97929	train-rmse:1.8035
[8]	eval-rmse:3.99634	train-rmse:1.66497
[9]	eval-rmse:4.00181	train-rmse:1.55703


In [519]:
size = len(preds)
errors = list()
for i in range(size):
    errors.append(preds[i]-labels[i])

In [520]:
errors_sq = [i**2 for i in errors]
np.sqrt(np.mean(errors_sq))

4.001806942187769

In [521]:
for i in range(size):
    print(preds[i], labels[i])

1.7090085 0.0
0.8596552 1.0
1.3128963 0.0
0.8596552 1.0
1.3128963 0.0
0.8596552 0.0
1.3128963 1.0
1.2545089 0.0
1.5010655 0.0
0.62581944 0.0
0.9145113 2.0
0.8325039 4.0
1.5010655 3.0
1.6042515 2.0
2.1032083 0.0
2.469134 3.0
3.4706154 3.0
6.3180327 4.0
7.0710053 4.0
8.47669 2.0
7.934303 2.0
10.573531 5.0
9.991907 8.0
13.211583 9.0
14.236577 11.0
11.488022 12.0
11.861576 13.0
13.867683 13.0
13.867683 13.0
12.943075 13.0
12.943075 13.0
14.30289 13.0
14.30289 13.0
14.30289 13.0
14.30289 13.0
14.495945 13.0
14.147023 13.0
15.927924 13.0
15.927924 13.0
14.871481 13.0
11.150859 5.0
3.8109791 1.0
2.0423324 0.0
0.54988295 0.0
0.50376225 2.0
0.73738927 0.0
1.7090085 0.0
1.419779 0.0
1.6245121 0.0
1.419779 0.0
1.6245121 5.0
1.419779 2.0
1.6245121 0.0
2.0832818 1.0
2.288015 0.0
2.271451 1.0
2.4761841 0.0
2.271451 0.0
2.4761841 0.0
2.271451 0.0
2.4761841 1.0
2.6211936 3.0
3.078327 0.0
3.1525288 2.0
3.9055018 4.0
3.759663 4.0
4.5126367 5.0
7.24561 2.0
7.3533826 2.0
9.824226 9.0
9.490188 11.0
14.8989

In [393]:
weather_2 = RDB.weather_times(RDB.read_weather(2))

In [394]:
weather_2

,apparentTemperature,cloudCover,dewPoint,humidity,icon,nearestStormBearing,nearestStormDistance,ozone,precipIntensity,precipProbability,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed,date,day,hour,minute,second,first_half_hour,second_half_hour
0,12.47,0.75,5.73,0.63,partly-cloudy-day,304.0,204.0,283.66,0.0,0.0,1032.54,Mostly Cloudy,12.47,1551095170,2.0,9.46,164.0,8.39,1.78,02/25/19,Mon,11,46,10,False,True
1,12.57,0.75,5.76,0.63,partly-cloudy-day,304.0,204.0,283.60,0.0,0.0,1032.55,Mostly Cloudy,12.57,1551095425,2.0,9.45,164.0,8.40,1.78,02/25/19,Mon,11,50,25,False,True
2,12.13,0.75,5.93,0.66,partly-cloudy-day,15.0,528.0,280.38,0.0,0.0,1030.71,Mostly Cloudy,12.13,1551112303,0.0,10.77,124.0,8.24,3.56,02/25/19,Mon,16,31,43,False,True
3,12.08,0.75,5.92,0.66,partly-cloudy-day,15.0,528.0,280.37,0.0,0.0,1030.72,Mostly Cloudy,12.08,1551112437,0.0,10.83,125.0,8.24,3.56,02/25/19,Mon,16,33,57,False,True
4,12.08,0.75,5.91,0.66,partly-cloudy-day,15.0,528.0,280.36,0.0,0.0,1030.72,Mostly Cloudy,12.08,1551112447,0.0,10.83,125.0,8.24,3.56,02/25/19,Mon,16,34,7,False,True
5,12.07,0.75,5.91,0.66,partly-cloudy-day,15.0,528.0,280.36,0.0,0.0,1030.72,Mostly Cloudy,12.07,1551112456,0.0,10.83,125.0,8.24,3.56,02/25/19,Mon,16,34,16,False,True
6,11.96,0.75,5.88,0.66,partly-cloudy-day,15.0,528.0,280.33,0.0,0.0,1030.75,Mostly Cloudy,11.96,1551112756,0.0,10.96,125.0,8.26,3.56,02/25/19,Mon,16,39,16,False,True
7,11.86,0.75,5.84,0.67,partly-cloudy-day,18.0,529.0,280.29,0.0,0.0,1030.79,Mostly Cloudy,11.86,1551113056,0.0,11.07,126.0,8.27,3.56,02/25/19,Mon,16,44,16,False,True
8,11.31,0.76,6.15,0.71,partly-cloudy-day,18.0,529.0,280.26,0.0,0.0,1030.82,Mostly Cloudy,11.31,1551113356,0.0,16.09,171.0,8.27,5.33,02/25/19,Mon,16,49,16,False,True
9,11.20,0.76,6.11,0.71,partly-cloudy-day,18.0,546.0,280.22,0.0,0.0,1030.85,Mostly Cloudy,11.20,1551113656,0.0,16.09,171.0,8.28,5.33,02/25/19,Mon,16,54,16,False,True


In [400]:
station_2.loc[0]['date']==weather_2.loc[0]['date']

False

In [409]:
%%time
for i in station_2.index:
    if station_2.loc[i]['date']==weather_2.loc[0]['date']:
        break
    elif station_2.loc[i]['date']<weather_2.loc[0]['date']:
        station_2.drop([i], inplace=True)

Wall time: 2.05 s


In [438]:
station_2

,status,bike_stands,available_bike_stands,available_bikes,last_updated,date,day,hour,minute,second,first_half_hour,second_half_hour
0,OPEN,20,19,1,1550592601,02/19/19,Tue,16,10,1,True,False
1,OPEN,20,20,0,1550593147,02/19/19,Tue,16,19,7,True,False
2,OPEN,20,19,1,1550593810,02/19/19,Tue,16,30,10,False,True
3,OPEN,20,19,1,1550594419,02/19/19,Tue,16,40,19,False,True
4,OPEN,20,20,0,1550595287,02/19/19,Tue,16,54,47,False,True
5,OPEN,20,20,0,1550595896,02/19/19,Tue,17,4,56,True,False
6,OPEN,20,19,1,1550596280,02/19/19,Tue,17,11,20,True,False
7,OPEN,20,19,1,1550596889,02/19/19,Tue,17,21,29,True,False
8,OPEN,20,19,1,1550597758,02/19/19,Tue,17,35,58,False,True
9,OPEN,20,20,0,1550597820,02/19/19,Tue,17,37,0,False,True


In [459]:
mergedStuff = pd.merge(station_2, weather_2, left_on=['date', 'hour', 'first_half_hour', 'second_half_hour'], right_on=['date', 'hour', 'first_half_hour', 'second_half_hour'], how='inner')
mergedStuff.drop_duplicates(subset='last_updated', inplace=True)
mergedStuff.drop_duplicates(subset='time', inplace=True)

mergedStuff.reset_index()

,index,status,bike_stands,available_bike_stands,available_bikes,last_updated,date,day_x,hour,minute_x,second_x,first_half_hour,second_half_hour,apparentTemperature,cloudCover,dewPoint,humidity,icon,nearestStormBearing,nearestStormDistance,ozone,precipIntensity,precipProbability,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed,day_y,minute_y,second_y
0,0,OPEN,20,17,3,1551094313,02/25/19,Mon,11,31,53,False,True,12.47,0.75,5.73,0.63,partly-cloudy-day,304.0,204.0,283.66,0.0,0.0,1032.54,Mostly Cloudy,12.47,1551095170,2.0,9.46,164.0,8.39,1.78,Mon,46,10
1,10,OPEN,20,16,4,1551112225,02/25/19,Mon,16,30,25,False,True,12.13,0.75,5.93,0.66,partly-cloudy-day,15.0,528.0,280.38,0.0,0.0,1030.71,Mostly Cloudy,12.13,1551112303,0.0,10.77,124.0,8.24,3.56,Mon,31,43
2,55,OPEN,20,16,3,1551114338,02/25/19,Mon,17,5,38,True,False,11.00,0.76,6.04,0.72,partly-cloudy-day,15.0,529.0,280.15,0.0,0.0,1030.91,Mostly Cloudy,11.00,1551114256,0.0,16.09,171.0,8.31,5.32,Mon,4,16
3,85,OPEN,20,16,3,1551116329,02/25/19,Mon,17,38,49,False,True,10.43,0.74,5.87,0.73,partly-cloudy-day,186.0,101.0,279.98,0.0,0.0,1031.05,Mostly Cloudy,10.43,1551116057,0.0,16.09,171.0,8.40,5.28,Mon,34,17
4,109,OPEN,20,18,1,1551118011,02/25/19,Mon,18,6,51,True,False,10.73,0.70,6.21,0.74,partly-cloudy-night,170.0,89.0,286.20,0.0,0.0,1031.39,Mostly Cloudy,10.73,1551117857,0.0,10.12,222.0,7.90,1.50,Mon,4,17
5,139,OPEN,20,16,3,1551119470,02/25/19,Mon,18,31,10,False,True,10.42,0.64,6.09,0.75,partly-cloudy-night,172.0,80.0,285.77,0.0,0.0,1031.40,Mostly Cloudy,10.42,1551119657,0.0,10.88,216.0,8.16,1.62,Mon,34,17
6,163,OPEN,20,18,2,1551121293,02/25/19,Mon,19,1,33,True,False,7.98,0.65,3.74,0.66,partly-cloudy-night,262.0,341.0,285.32,0.0,0.0,1031.38,Mostly Cloudy,9.88,1551121458,0.0,10.15,151.0,8.44,3.76,Mon,4,18
7,187,OPEN,20,15,5,1551123089,02/25/19,Mon,19,31,29,False,True,7.55,0.63,3.68,0.67,partly-cloudy-night,180.0,77.0,284.78,0.0,0.0,1030.88,Mostly Cloudy,9.56,1551123258,0.0,10.91,156.0,8.78,3.82,Mon,34,18
8,217,OPEN,20,17,3,1551125366,02/25/19,Mon,20,9,26,True,False,6.39,0.68,5.62,0.81,partly-cloudy-night,209.0,63.0,284.26,0.0,0.0,1030.74,Mostly Cloudy,8.61,1551125058,0.0,10.17,157.0,9.08,3.82,Mon,4,18
9,241,OPEN,20,13,7,1551126798,02/25/19,Mon,20,33,18,False,True,6.19,0.63,5.52,0.82,partly-cloudy-night,291.0,251.0,283.83,0.0,0.0,1030.69,Mostly Cloudy,8.44,1551126859,0.0,10.93,160.0,9.20,3.81,Mon,34,19


In [476]:
station = RDB.add_times(RDB.create_station_dictionary(2)[2])
weather = RDB.weather_times(RDB.read_weather(2))
df = pd.merge(station, weather, left_on=['date', 'hour', 'first_half_hour', 'second_half_hour'], right_on=['date', 'hour', 'first_half_hour', 'second_half_hour'], how='inner')
df.drop_duplicates(subset='last_updated', inplace=True)
df.drop_duplicates(subset='time', inplace=True)

df = df.reset_index()
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
weather_icons = ['partly-cloudy-day',
                 'partly-cloudy-night',
                 'clear-night',
                 'clear-day',
                 'fog',
                 'wind',
                 'cloudy',
                 'rain'
                ]
for day in days:
    df[day] = 0

for icon in weather_icons:
    df[icon] = 0

for j in df.index:
    val = df.loc[j, 'day_x']
    for day in days:
        if val == day:
            df.loc[j, day] = 1

In [477]:
df

,index,status,bike_stands,available_bike_stands,available_bikes,last_updated,date,day_x,hour,minute_x,second_x,first_half_hour,second_half_hour,apparentTemperature,cloudCover,dewPoint,humidity,icon,nearestStormBearing,nearestStormDistance,ozone,precipIntensity,precipProbability,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed,day_y,minute_y,second_y,Mon,Tue,Wed,Thu,Fri,Sat,Sun,partly-cloudy-day,partly-cloudy-night,clear-night,clear-day,fog,wind,cloudy,rain
0,0,OPEN,20,17,3,1551094313,02/25/19,Mon,11,31,53,False,True,12.47,0.75,5.73,0.63,partly-cloudy-day,304.0,204.0,283.66,0.0,0.0,1032.54,Mostly Cloudy,12.47,1551095170,2.0,9.46,164.0,8.39,1.78,Mon,46,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10,OPEN,20,16,4,1551112225,02/25/19,Mon,16,30,25,False,True,12.13,0.75,5.93,0.66,partly-cloudy-day,15.0,528.0,280.38,0.0,0.0,1030.71,Mostly Cloudy,12.13,1551112303,0.0,10.77,124.0,8.24,3.56,Mon,31,43,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,55,OPEN,20,16,3,1551114338,02/25/19,Mon,17,5,38,True,False,11.00,0.76,6.04,0.72,partly-cloudy-day,15.0,529.0,280.15,0.0,0.0,1030.91,Mostly Cloudy,11.00,1551114256,0.0,16.09,171.0,8.31,5.32,Mon,4,16,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,85,OPEN,20,16,3,1551116329,02/25/19,Mon,17,38,49,False,True,10.43,0.74,5.87,0.73,partly-cloudy-day,186.0,101.0,279.98,0.0,0.0,1031.05,Mostly Cloudy,10.43,1551116057,0.0,16.09,171.0,8.40,5.28,Mon,34,17,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,109,OPEN,20,18,1,1551118011,02/25/19,Mon,18,6,51,True,False,10.73,0.70,6.21,0.74,partly-cloudy-night,170.0,89.0,286.20,0.0,0.0,1031.39,Mostly Cloudy,10.73,1551117857,0.0,10.12,222.0,7.90,1.50,Mon,4,17,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,139,OPEN,20,16,3,1551119470,02/25/19,Mon,18,31,10,False,True,10.42,0.64,6.09,0.75,partly-cloudy-night,172.0,80.0,285.77,0.0,0.0,1031.40,Mostly Cloudy,10.42,1551119657,0.0,10.88,216.0,8.16,1.62,Mon,34,17,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,163,OPEN,20,18,2,1551121293,02/25/19,Mon,19,1,33,True,False,7.98,0.65,3.74,0.66,partly-cloudy-night,262.0,341.0,285.32,0.0,0.0,1031.38,Mostly Cloudy,9.88,1551121458,0.0,10.15,151.0,8.44,3.76,Mon,4,18,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,187,OPEN,20,15,5,1551123089,02/25/19,Mon,19,31,29,False,True,7.55,0.63,3.68,0.67,partly-cloudy-night,180.0,77.0,284.78,0.0,0.0,1030.88,Mostly Cloudy,9.56,1551123258,0.0,10.91,156.0,8.78,3.82,Mon,34,18,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,217,OPEN,20,17,3,1551125366,02/25/19,Mon,20,9,26,True,False,6.39,0.68,5.62,0.81,partly-cloudy-night,209.0,63.0,284.26,0.0,0.0,1030.74,Mostly Cloudy,8.61,1551125058,0.0,10.17,157.0,9.08,3.82,Mon,4,18,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,241,OPEN,20,13,7,1551126798,02/25/19,Mon,20,33,18,False,True,6.19,0.63,5.52,0.82,partly-cloudy-night,291.0,251.0,283.83,0.0,0.0,1030.69,Mostly Cloudy,8.44,1551126859,0.0,10.93,160.0,9.20,3.81,Mon,34,19,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
